In [18]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog
import pandas as pd

class NewsClassifier:
    def __init__(self):
        self.classifier = None
        self.vectorizer = None
        
    def preprocess(self, text):
        # Remove punctuation
        text = text.translate(str.maketrans("", "", string.punctuation))
        
        # Tokenize the text
        tokens = nltk.word_tokenize(text.lower())
        
        # Remove stop words and stem the words
        stemmer = PorterStemmer()
        stop_words = set(stopwords.words('english'))
        stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
        
        # Return the preprocessed text as a space-separated string
        return ' '.join(stemmed_tokens)
    
    def train_model(self, X, y):
        # Vectorize the input text data
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(X)
        
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Train a Naive Bayes classifier
        classifier = MultinomialNB()
        classifier.fit(X_train, y_train)
        
        self.classifier = classifier
        self.vectorizer = vectorizer
        
        # Evaluate the model
        self.evaluate_model(X_test, y_test)
    
    def evaluate_model(self, X_test, y_test):
        # Predict the labels for the test dataset
        y_pred = self.classifier.predict(X_test)
        
        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        messagebox.showinfo("Model Evaluation", f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1-score: {f1}")
    
    def predict_category(self, text):
        preprocessed_text = self.preprocess(text)
        text_vector = self.vectorizer.transform([preprocessed_text])
        predicted_category = self.classifier.predict(text_vector)[0]
        messagebox.showinfo("Category Prediction", f"The predicted category is: {predicted_category}")

    def browse_file(self):
        file_path = filedialog.askopenfilename(filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
        if file_path:
            try:
                df = pd.read_csv(file_path)
                category_column = 'Category'
                text_column = 'Text'
                categories = df[category_column].values
                texts = df[text_column].values

                X = [self.preprocess(text) for text in texts]
                y = categories

                self.train_model(X, y)
            except Exception as e:
                messagebox.showerror("Error", str(e))
    
    def classify_text(self):
        text = text_entry.get("1.0", tk.END).strip()
        if text:
            self.predict_category(text)

# Create an instance of the NewsClassifier class
news_classifier = NewsClassifier()

# Create the main GUI window
window = tk.Tk()
window.title("News Article Classifier")

# Create a label for the text entry
text_label = tk.Label(window, text="Enter news article:")
text_label.pack()

# Create a text entry field
text_entry = tk.Text(window, height=10, width=50)
text_entry.pack()

# Create a button to classify the text
classify_button = tk.Button(window, text="Classify", command=news_classifier.classify_text)
classify_button.pack()

# Create a button to browse and load a CSV file
browse_button = tk.Button(window, text="Browse", command=news_classifier.browse_file)
browse_button.pack()

# Run the main GUI loop
window.mainloop()

